## Project: Watching the Stock Market

### Objective

You are asked by a company to help them make more informed decisions on investments. To start, you will be watching the stock market, collecting data, and identifying trends!

### Hosting your project

Using DB Browser for SQLite as a visual tool for working with SQLite databases.

- SQLite can store an entire database in a single file, which usually has a .sqlite or .db extension. To open a database, select Open Database at the top of the window and browse for the file. Alternatively, you can choose to create a New Database by saving a file with the .sqlite or .db extension.
- To import data from a CSV file into a table, select “File > Import > Table from CSV file…” and browse for the CSV file. (Note: All fields imported from the CSV file will have a data type of TEXT. Be sure to convert fields to numeric type as needed)

### Basic Requirements

Let’s break this project down into a couple different parts.

Manipulation: Collect data on your pick of 5 stocks.

- Create a table called stocks, where you will be inserting your data.
- The stocks table should have a column for symbol, name, datetime, and price.
- Collect your data! Choose 3 times throughout the day to document the price of each stock and continue for at least 1 week. You can do this moving forward, or just take a retroactive look at the stock market by taking data historically from regular intervals (e.g. the first of the month for the last six months).

### Importing Data

In order to import the data, I have used yahoo.finance to extract the daily historical prices from 25-01-2019 to 24-01-2020 of the following stocks:
- Tesla Inc
- United Airlines Holdings Inc
- Lam Research Corp
- IDEXX Laboratories Inc
- Starbucks Corp

Then using the below python code I have converted the file to get the columns we are using in this project: symbol, name, datetime and price

```Python
import pandas as pd

tesla = pd.read_csv("TSLA.csv")
ual = pd.read_csv("UAL.csv")
lrcx = pd.read_csv("LRCX.csv")
idxx = pd.read_csv("IDXX.csv")
sbux = pd.read_csv("SBUX.csv")

name_tesla = ['TSLA:NASDAQ', 'Tesla Inc']
name_ual = ["UAL:NASDAQ", "United Airlines Holdings Inc"]
name_lrcx = ['LRCX:NASDAQ', 'Lam Research Corp']
name_idxx = ['IDXX:NASDAQ', "IDEXX Laboratories Inc"]
name_sbux = ["SBUX:NASDAQ", "Starbucks Corp"]

def format_data(dataframe, name):
    dataframe['symbol'], dataframe['name'] = name
    columns_order = ["symbol", "name", "Date", "Adj Close"]
    dataframe = dataframe[columns_order]
    return dataframe
    
tesla = format_data(tesla, name_tesla)
ual = format_data(ual, name_ual)
lrcx = format_data(lrcx, name_lrcx)
idxx = format_data(idxx, name_idxx)
sbux = format_data(sbux, name_sbux)

stocks_data = pd.concat([tesla, ual, lrcx, idxx, sbux]).reset_index(drop=True)
stocks_data.rename(columns={"Date": "datetime", "Adj Close": "price"}, inplace=True)

export_csv = stocks_data.to_csv("stocks_data.csv", index=False)
```

Then I have imported the data using DB Browser into our SQL database

### Perform basic analysis on the data and identify trends.

#### What are the distinct stocks in the table?

```mysql
SELECT DISTINCT name
FROM stocks;
```
![stock01.png](images/stock01.png)

#### Query all data for a single stock. Do you notice any overall trends?



```mysql
SELECT *
FROM stocks
WHERE symbol = "TSLA:NASDAQ";
```
![stock02.png](images/stock02.png)
![stock03.png](images/stock03.png)

Checking the data from Tesla, we can observe a bullish trend from 297 dollars at the beginning of the observation to 564 on the last closing date

#### Which rows have a price above 100? 

```mysql
SELECT *
FROM stocks
WHERE price > 100;
```

We observe that there are 3 stocks where more than 1 row with the price over 100:

Examples:

TSLA:NASDAQ	Tesla Inc	2019-01-25	297.040009

LRCX:NASDAQ	Lam Research Corp	2019-08-19	206.121902

IDXX:NASDAQ	IDEXX Laboratories Inc	2019-04-11	223.759995

#### Which rows have a price between 50 to 100?

```mysql
SELECT *
FROM stocks
WHERE price BETWEEN 50 AND 100;
```


We observe that there are 2 stocks where more than 1 row with the price between 50 to 100:

Examples:

UAL:NASDAQ	United Airlines Holdings Inc	2019-01-25	84.139999

SBUX:NASDAQ	Starbucks Corp	2020-01-22	92.529999


#### Sort the table by price. What are the minimum and maximum prices?

```mysql
SELECT *
FROM stocks
ORDER BY price;

SELECT *,
MIN(price)
FROM stocks;

SELECT *,
MAX(price)
FROM stocks;
```

We can see that the minimum price is Starbucks:

![stock04.png](images/stock04.png)

And the maxim price is Tesla:

![stock05.png](images/stock05.png)

It would be more usefull if we know the max and minimum price of each stock:

```mysql
SELECT symbol,
name,
datetime,
min(price)
FROM stocks
GROUP BY symbol;

SELECT symbol,
name,
datetime,
max(price)
FROM stocks
GROUP BY symbol;
```

![stock06.png](images/stock06.png)
![stock07.png](images/stock07.png)

#### Percentage of change from minimum to maximum of each stock

We can easly know the hypotetical maximum performance we could have got of each stock on this dataset:

```mysql
SELECT symbol,
name,
(max(price)-min(price))/min(price)*100 performance
FROM stocks
GROUP BY symbol
ORDER BY performance DESC;
```

![stock08.png](images/stock08.png)

We can see that the stock with the "best performance" from min to max is Tesla, with a increase of almost 220%